In [41]:
from bs4 import BeautifulSoup
import requests
import re
import csv

In [42]:
def get_html(url):
    response = requests.get(url)
    if not response.ok:
        print(f"{response.status_code}, url: {url}")
    return response.text

def get_games(html):
    soup = BeautifulSoup(html, 'lxml')
    pattern = r'^https://store.steampowered.com/app/'
    games = soup.find_all('a', href=re.compile(pattern))
    return games

def get_hover_data(id):
    url = f"https://store.steampowered.com/apphoverpublic/{id}"
    html = get_html(url)
    soup = BeautifulSoup(html, 'lxml')

    try:
        title = soup.find('h4', class_='hover_title').text.strip()
    except:
        title = 'Not Define'
        print(url)

    try:
        released = soup.find('div', class_='hover_release').span.text.split(':')[-1].strip()
    except:
        released = 'Not Define'
        print(url)

    try:
        review_raw = soup.find('div', class_='hover_review_summary').text.strip()
    except:
        review_raw = 'Not Define'
        reviews = ''
        print(url)
    else:
        pattern = r'\d+'
        reviews = int("".join(re.findall(pattern, review_raw)))

    try:
        tags_raw = soup.find_all('div', class_='app_tag')
    except:
        tags = 'Not Define'
        print(url)
    else:
        tags_text = [tag.text for tag in tags_raw]
        tags = ', '.join(tags_text)

        data = {}
        data['title'] = title
        data['released'] = released
        data['reviews'] = reviews
        data['tags'] = tags 

    write_csv(data=data)

def write_csv(data):
    with open('result.csv', 'a') as file:
        fields = ['title', 'reviews', 'released', 'tags']
        writer = csv.DictWriter(file, fieldnames=fields)
        writer.writerow(data)


In [43]:
#link https://store.steampowered.com/app/1295510/DRAGON_QUEST_XI_S__Les_Combattants_de_la_destine__dition_ultime/?snr=1_7_7_240_150_2
#link https://store.steampowered.com/search/results/?query&start=50&count=50&dynamic_data=&tags=1702&snr=1_7_7_240_7&infinite=1
# link https://store.steampowered.com/apphoverpublic/346110

In [ ]:
start = 0 
all_games = []
url = f"https://store.steampowered.com/search/results/?query&start={start}&count=50&dynamic_data=&tags=1702&snr=1_7_7_240_7"
while True:
    games = get_games(get_html(url))
    if games:
        all_games.extend(games)
        start += 300
        url = f"https://store.steampowered.com/search/results/?query&start={start}&count=50&dynamic_data=&tags=1702&snr=1_7_7_240_7"
    else:
        break


print(len(all_games))


for game in all_games:
    id = game.get('data-ds-appid')
    get_hover_data(id=id)
